In [1]:
import os
import sys

# Get the path to the directory containing the current notebook
# This assumes agentic_rag.ipynb is directly inside src/agentic_rag/
current_notebook_dir = os.path.abspath(os.path.dirname('__file__'))

# Navigate up to the project root (AGENTIC_RAG/)
# current_notebook_dir is `AGENTIC_RAG/src/agentic_rag/`
# Go up once to `AGENTIC_RAG/src/`
# Go up twice to `AGENTIC_RAG/` (the root)
project_root = os.path.abspath(os.path.join(current_notebook_dir, '..', '..'))

# Add the project root to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Project root added to sys.path: {project_root}")
print(f"Current sys.path: {sys.path}")

Project root added to sys.path: /Users/avinashgohite/Desktop/agentic_RAG
Current sys.path: ['/Users/avinashgohite/Desktop/agentic_RAG', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/avinashgohite/Desktop/agentic_RAG/.venv/lib/python3.11/site-packages']


In [2]:
!pip  install "chonkie[semantic]"
!pip  install markitdown
!pip  install qdrant-client
!pip  install fastembed


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task
from crewai_tools import SerperDevTool
from crewai_tools import PDFSearchTool
from src.agentic_rag.tool.custom_tool import DocumentSearchTool
# If FireCrawlWebSearchTool is also in custom_tool.py and needed here:
from src.agentic_rag.tool.custom_tool import FireCrawlWebSearchTool


In [4]:
pdf_tool = DocumentSearchTool(file_path='/Users/avinashgohite/Desktop/agentic_RAG/knowledge/dspy.pdf')

In [5]:
web_search_tool = SerperDevTool()

In [6]:
import yaml
with open('config/agents.yaml', 'r') as f:
    agents_config = yaml.safe_load(f)

with open('config/tasks.yaml', 'r') as f:
    tasks_config = yaml.safe_load(f)

In [7]:
retriever_agent = Agent(
    config=agents_config['retriever_agent'],
    verbose=False,
    tools=[
        pdf_tool,
        web_search_tool
    ]
)

response_synthesizer_agent = Agent(
    config=agents_config['response_synthesizer_agent'],
    verbose=False
)

retrieval_task = Task(
    description=tasks_config['retrieval_task']['description'],
    expected_output=tasks_config['retrieval_task']['expected_output'],  
    agent=retriever_agent
)

response_task = Task(
    description=tasks_config['response_task']['description'],
    expected_output=tasks_config['response_task']['expected_output'], 
    agent=response_synthesizer_agent
)

In [8]:
crew = Crew(
			agents=[retriever_agent, response_synthesizer_agent], 
			tasks=[retrieval_task, response_task],
			process=Process.sequential,
			verbose=False,
			# process=Process.hierarchical, # In case you wanna use that instead https://docs.crewai.com/how-to/Hierarchical/
		)

In [9]:
result = crew.kickoff(inputs={"query": "Who is elon musk?"})
print(result)

Elon Reeve Musk is a prominent businessman renowned for his impactful leadership in several high-profile technology companies. Born on June 28, 1971, in Pretoria, South Africa, Musk has significantly influenced sectors like electric vehicles and space exploration. He is notably associated with his leadership roles in Tesla and SpaceX and has founded other companies such as OpenAI. As of May 2025, Forbes estimates his net worth to be 405.8 billion USD, maintaining his status as the wealthiest person globally since 2021 according to the Google knowledge graph. His diverse family life includes several children and marriages to Talulah Riley and Justine Musk. Additionally, Musk has served as an advisor to the President of the United States. More information can be found on his [Wikipedia page](https://en.wikipedia.org/wiki/Elon_Musk).
